# IA25_P01_G18 - Class Timetable

## Introdução
O objetivo deste projeto é desenvolver um agente inteligente para a geração de horários de aulas,formulado como um problema de satisfação de restrições (Constraint Satisfaction Problem - CSP).


### Equipa
- André Ferreira (a20764)

## Carregar dataset e construir estruturas

In [139]:
from pathlib import Path

DATA_PATH = Path("data/ClassTT_02_real") 
raw = DATA_PATH.read_text(encoding="utf-8", errors="ignore").splitlines()

def parse_block(lines, header_prefix):
    try:
        start = next(i for i, l in enumerate(lines) if l.strip().startswith(header_prefix))
    except StopIteration:
        return []
    body = []
    for l in lines[start+1:]:
        if l.strip().startswith("#"):
            break
        if l.strip():
            body.append(l.strip())
    return body

# Blocos
cc   = parse_block(raw, "#cc")     
olw  = parse_block(raw, "#olw")    
dsd  = parse_block(raw, "#dsd")    
tr   = parse_block(raw, "#tr")     
rr   = parse_block(raw, "#rr")     
oc   = parse_block(raw, "#oc")     
rms  = parse_block(raw, "#rooms")  

# classes -> cursos
COURSES_BY_CLASS = {}
for row in cc:
    parts = row.split()
    cls, courses = parts[0], parts[1:]
    COURSES_BY_CLASS[cls] = courses

CLASSES = sorted(COURSES_BY_CLASS.keys())
ALL_COURSES = sorted({c for lst in COURSES_BY_CLASS.values() for c in lst})

# cursos com 1 aula/semana
ONE_LESSON = set()
for row in olw:
    parts = row.split()
    for c in parts:  
        if c in ALL_COURSES:
            ONE_LESSON.add(c)

# docente -> cursos  e curso -> docente
TEACHER_COURSES = {}
TEACHER_BY_COURSE = {}
for row in dsd:
    parts = row.split()
    t, cs = parts[0], parts[1:]
    TEACHER_COURSES[t] = cs
    for c in cs:
        TEACHER_BY_COURSE[c] = t

# indisponibilidades por docente
UNAVAILABLE = {}
for row in tr:
    parts = row.split()
    t, slots = parts[0], list(map(int, parts[1:]))
    UNAVAILABLE[t] = set(slots)

# sala obrigatória por curso
ROOM_REQUIRED = {}
for row in rr:
    c, r = row.split()
    ROOM_REQUIRED[c] = r

# aulas online por índice
ONLINE_IDX = {}
for row in oc:
    c, idx = row.split()
    if c in ALL_COURSES:
        ONLINE_IDX.setdefault(c, set()).add(int(idx))

# salas + ONLINE
ROOMS_ALL = [rms[i].split()[0] for i in range(len(rms))]  # uma sala por linha
if "ONLINE" not in ROOMS_ALL:
    ROOMS_ALL.append("ONLINE")

SLOTS = list(range(1, 21))  # 5 dias × 4 blocos = 20

def day_of(slot: int) -> int: return (slot - 1) // 4
def block_in_day(slot: int) -> int: return (slot - 1) % 4

missing_teachers = [c for c in ALL_COURSES if c not in TEACHER_BY_COURSE]
if missing_teachers:
    print("\nCursos sem professor definido no #dsd:", missing_teachers)

# professores e as UC associadas
print("\nProfessores e UCs associadas:")
for t, cs in TEACHER_COURSES.items():
    print(f"  {t}: {', '.join(cs)}")

# indisponibilidades
print("\nIndisponibilidades (slots bloqueados):")
for t, slots in UNAVAILABLE.items():
    if slots:
        print(f"  {t}: {sorted(slots)}")
    else:
        print(f"  {t}: nenhuma")

print("\nTurmas:", CLASSES)
print("Cursos:", ALL_COURSES)
print("Salas:", ROOMS_ALL)
print("UCs com 1 aula/semana:", sorted(ONE_LESSON))



Professores e UCs associadas:
  João: PDM, PS, IM, GSI
  Pedro: ISI, R, IA
  António: IA, SETR, IE, RCE
  Manuel: PA, RCSD, AST, ISC, GUS
  Ana: IAAJ, PAG
  Isabel: PVR, TCGEV

Indisponibilidades (slots bloqueados):
  Isabel: [13, 14, 15, 16, 17, 18, 19, 20]
  Manuel: [1, 2, 3, 4]
  João: [9, 10, 11, 12, 17, 18, 19, 20]

Turmas: ['LDJG', 'LEEC', 'LEIM', 'LESI']
Cursos: ['AST', 'GSI', 'GUS', 'IA', 'IAAJ', 'IE', 'IM', 'ISC', 'ISI', 'PA', 'PAG', 'PDM', 'PS', 'PVR', 'R', 'RCE', 'RCSD', 'SETR', 'TCGEV']
Salas: ['SalaT', 'SalaC', 'Lab01', 'ONLINE']
UCs com 1 aula/semana: []


## CSP: variáveis e domínios

In [140]:
from constraint import Problem, MinConflictsSolver, AllDifferentConstraint

problem = Problem(MinConflictsSolver())

def lesson_indexes(course):
    return [1] if course in ONE_LESSON else [1, 2]

for c in ALL_COURSES:
    teacher = TEACHER_BY_COURSE[c]
    blocked = UNAVAILABLE.get(teacher, set())
    allowed_slots = [s for s in SLOTS if s not in blocked]  # pré-poda

    for i in lesson_indexes(c):
        
        problem.addVariable(f"{c}_{i}", allowed_slots)

        if c in ROOM_REQUIRED:
            problem.addVariable(f"{c}_{i}_room", [ROOM_REQUIRED[c]])
        elif c in ONLINE_IDX and i in ONLINE_IDX[c]:
            problem.addVariable(f"{c}_{i}_room", ["ONLINE"])
        else:
            problem.addVariable(f"{c}_{i}_room", ROOMS_ALL)


## Restrições

In [141]:
# Online obrigatório quando marcado
for c, idxs in ONLINE_IDX.items():
    for i in idxs:
        if i in lesson_indexes(c):  # só se a UC tiver esse índice
            problem.addConstraint(lambda r: r == "ONLINE", (f"{c}_{i}_room",))

# Proibir ONLINE quando não estiver marcado
for c in ALL_COURSES:
    for i in lesson_indexes(c):
        if not (c in ONLINE_IDX and i in ONLINE_IDX[c]):
            problem.addConstraint(lambda r: r != "ONLINE", (f"{c}_{i}_room",))

# Sala obrigatória (reforço)
for c, room in ROOM_REQUIRED.items():
    for i in lesson_indexes(c):
        problem.addConstraint(lambda r, req=room: r == req, (f"{c}_{i}_room",))

# Não sobrepor DOCENTE
for t, courses in TEACHER_COURSES.items():
    slot_vars = [f"{c}_{i}" for c in courses for i in lesson_indexes(c)]
    if len(slot_vars) > 1:
        problem.addConstraint(AllDifferentConstraint(), slot_vars)

# Não sobrepor TURMA
for cls, courses in COURSES_BY_CLASS.items():
    slot_vars = [f"{c}_{i}" for c in courses for i in lesson_indexes(c)]
    if len(slot_vars) > 1:
        problem.addConstraint(AllDifferentConstraint(), slot_vars)

# Máximo 3 aulas por dia por TURMA
def add_max3_lessons_per_day_for_class(problem, cls, courses):
    slot_vars = [f"{c}_{i}" for c in courses for i in lesson_indexes(c)]
    def ok_max3_per_day(*slots):
        per_day = {d: 0 for d in range(5)}
        for s in slots:
            d = day_of(s)
            per_day[d] += 1
            if per_day[d] > 3:
                return False
        return True
    if slot_vars:
        problem.addConstraint(ok_max3_per_day, tuple(slot_vars))

for cls, courses in COURSES_BY_CLASS.items():
    add_max3_lessons_per_day_for_class(problem, cls, courses)

# Se houver aulas online numa turma, no mesmo dia e ≤3
def online_same_day_max3_for_class(problem, cls, courses):
    online_vars = []
    for c in courses:
        for i in (ONLINE_IDX.get(c) or []):
            if i in lesson_indexes(c):
                online_vars.append(f"{c}_{i}")
    if len(online_vars) <= 1:
        return
    def same_day_max3(*slots):
        days = [day_of(s) for s in slots]
        return (len(set(days)) == 1) and (len(slots) <= 3)
    problem.addConstraint(same_day_max3, tuple(online_vars))

for cls, courses in COURSES_BY_CLASS.items():
    online_same_day_max3_for_class(problem, cls, courses)


## Tempo a encontrar a primeira solução

In [142]:
import time

start = time.perf_counter()
solution = problem.getSolution() 
elapsed = time.perf_counter() - start

if solution:
    print(f"Solução encontrada em {elapsed:.3f}s")
else:
    print("Nenhuma solução encontrada")


Solução encontrada em 0.009s


## Impressão da solução em tabela

In [143]:
def print_solution_table(sol, courses):
    print(f"{'Curso':<8} | {'Aula(slot,sala)':<24} | {'Aula2(slot,sala)':<24}")
    print("-"*70)
    for c in courses:
        idxs = lesson_indexes(c)
        s1 = sol[f"{c}_1"]; r1 = sol.get(f"{c}_1_room", "")
        if len(idxs) == 1:
            print(f"{c:<8} | ({s1:>2}, {r1:<12}) | {'-':<24}")
        else:
            s2 = sol[f"{c}_2"]; r2 = sol.get(f"{c}_2_room", "")
            print(f"{c:<8} | ({s1:>2}, {r1:<12}) | ({s2:>2}, {r2:<12})")


## Função de pontuação (soft constraints)

In [144]:
from collections import defaultdict

def score_solution(sol) -> dict:
    
    def day(s): return (s-1)//4
    def block(s): return (s-1)%4

    score = 0
    detail = {}

    # UC em dias distintos
    uc_distintos = 0
    for c in ALL_COURSES:
        if day(sol[f"{c}_1"]) != day(sol[f"{c}_2"]):
            uc_distintos += 1
    detail["uc_dias_distintos"] = uc_distintos
    score += uc_distintos

    # Agrupar por turma
    turma_dias = {cls: defaultdict(list) for cls in COURSES_BY_CLASS}
    turma_salas = {cls: set() for cls in COURSES_BY_CLASS}

    for cls, courses in COURSES_BY_CLASS.items():
        for c in courses:
            for i in (1,2):
                s = sol[f"{c}_{i}"]
                turma_dias[cls][day(s)].append(block(s))
                r = sol.get(f"{c}_{i}_room")
                if r and r != "ONLINE":
                    turma_salas[cls].add(r)

    # Ideal 4 dias/semana
    dias_ideais_score = 0
    dias_ideais_penal = 0
    for cls, by_day in turma_dias.items():
        used_days = sum(1 for d, slots in by_day.items() if slots)
        if used_days <= 4:
            dias_ideais_score += 1
        else:
            dias_ideais_penal += (used_days - 4)
    detail["turma_dias_<=4_bonus"] = dias_ideais_score
    detail["turma_dias_excesso_penal"] = -dias_ideais_penal
    score += dias_ideais_score - dias_ideais_penal

    # Consecutividade
    consecutivos = 0
    for cls, by_day in turma_dias.items():
        for d, blocks in by_day.items():
            blocks.sort()
            consecutivos += sum(1 for a,b in zip(blocks, blocks[1:]) if b-a == 1)
    detail["consecutivos"] = consecutivos
    score += consecutivos

    # Minimizar nº de salas diferentes por turma
    salas_penal = 0
    for cls, rooms in turma_salas.items():
        if len(rooms) > 1:
            salas_penal += (len(rooms)-1)
    detail["min_salas_penal"] = -salas_penal
    score -= salas_penal

    detail["total"] = score
    return detail


## Builder só de SLOTS

In [145]:
from constraint import Problem, MinConflictsSolver, AllDifferentConstraint
import time
import random

def lesson_indexes(c):
    return [1] if c in ONE_LESSON else [1, 2]

def build_problem_slots_only():
    p = Problem(MinConflictsSolver())

    for c in ALL_COURSES:
        t = TEACHER_BY_COURSE[c]
        allowed = [s for s in SLOTS if s not in UNAVAILABLE.get(t, set())]
        random.shuffle(allowed)
        for i in lesson_indexes(c):
            p.addVariable(f"{c}_{i}", allowed)

    # não sobrepor por docente
    for t, courses in TEACHER_COURSES.items():
        vars_ = [f"{c}_{i}" for c in courses for i in lesson_indexes(c)]
        if len(vars_) > 1:
            p.addConstraint(AllDifferentConstraint(), vars_)

    # não sobrepor por turma
    for cls, courses in COURSES_BY_CLASS.items():
        vars_ = [f"{c}_{i}" for c in courses for i in lesson_indexes(c)]
        if len(vars_) > 1:
            p.addConstraint(AllDifferentConstraint(), vars_)

    # máx 3 aulas/dia por turma
    def add_max3_for_class(courses):
        vars_ = [f"{c}_{i}" for c in courses for i in lesson_indexes(c)]
        def ok(*slots):
            cnt = {d: 0 for d in range(5)}
            for s in slots:
                d = (s-1)//4
                cnt[d] += 1
                if cnt[d] > 3:
                    return False
            return True
        if vars_:
            p.addConstraint(ok, tuple(vars_))
    for cls, courses in COURSES_BY_CLASS.items():
        add_max3_for_class(courses)


    return p


## Alocar SALAS (greedy) + detetar conflitos

In [146]:
from collections import defaultdict

def allocate_rooms_greedy(slot_sol):
    sol = dict(slot_sol)
    rooms_phys = [r for r in ROOMS_ALL if r != "ONLINE"]

    COURSE_CLASS = {}
    for cls, courses in COURSES_BY_CLASS.items():
        for c in courses:
            COURSE_CLASS[c] = cls

    for c in ALL_COURSES:
        for i in lesson_indexes(c):
            if c in ONLINE_IDX and i in ONLINE_IDX[c]:
                sol[f"{c}_{i}_room"] = "ONLINE"
            elif c in ROOM_REQUIRED:
                sol[f"{c}_{i}_room"] = ROOM_REQUIRED[c]

    per_slot = defaultdict(list)
    for c in ALL_COURSES:
        for i in lesson_indexes(c):
            s = sol[f"{c}_{i}"]
            per_slot[s].append((c, i))

    preferred_room_by_class = defaultdict(lambda: None)

    for s, items in per_slot.items():
        taken = {sol.get(f"{c}_{i}_room") for c,i in items if sol.get(f"{c}_{i}_room") not in (None, "ONLINE")}
        taken.discard(None)

        for c, i in items:
            if sol.get(f"{c}_{i}_room") in (None,):
                cls = COURSE_CLASS[c]
                pref = preferred_room_by_class[cls]
                if pref and pref not in taken and pref in rooms_phys:
                    sol[f"{c}_{i}_room"] = pref
                    taken.add(pref)

        free = [r for r in rooms_phys if r not in taken]
        for c, i in items:
            if sol.get(f"{c}_{i}_room") in (None,):
                if free:
                    chosen = free.pop(0)
                    sol[f"{c}_{i}_room"] = chosen
                    taken.add(chosen)
                else:
                    sol[f"{c}_{i}_room"] = rooms_phys[0]

        for c, i in items:
            r = sol[f"{c}_{i}_room"]
            if r not in (None, "ONLINE"):
                preferred_room_by_class[COURSE_CLASS[c]] = r

    return sol

def room_conflicts(sol):
    used = {}
    conf = []
    for c in ALL_COURSES:
        for i in lesson_indexes(c):
            s = sol[f"{c}_{i}"]; r = sol[f"{c}_{i}_room"]
            if r == "ONLINE": 
                continue
            key = (s, r)
            if key in used:
                conf.append((key, used[key], f"{c}_{i}"))
            else:
                used[key] = f"{c}_{i}"
    return conf


## Procurar várias soluções e escolher a melhor

In [147]:
def pick_best_solution_slots_only(max_restarts=10, per_run_timeout_sec=1.0):
   
    best, best_detail = None, None
    checked = 0
    t_start = time.perf_counter()

    for _ in range(max_restarts):
        p_slots = build_problem_slots_only()
        t0 = time.perf_counter()
        sol_slots = p_slots.getSolution()  
        dt = time.perf_counter() - t0

        if not sol_slots:
            continue

        full = allocate_rooms_greedy(sol_slots)   
        if room_conflicts(full):                  
            continue

        d = score_solution(full)
        checked += 1
        if best is None or d["total"] > (best_detail or {}).get("total", -10**9):
            best, best_detail = full, d

        # limite de tempo por tentativa
        if dt > per_run_timeout_sec:
            break

    elapsed = time.perf_counter() - t_start
    return best, best_detail, checked, elapsed

best, detail, nchecked, elapsed = pick_best_solution_slots_only(max_restarts=12, per_run_timeout_sec=1.2)
print(f"Tentativas válidas: {nchecked}  |  Tempo total: {elapsed:.2f}s")
if best:
    print("Score:", detail)
    print_solution_table(best, ALL_COURSES)
    print("Conflitos de sala:", "Nenhum" if not room_conflicts(best) else room_conflicts(best))
else:
    print("Não foi possível obter soluções no tempo dado.")


Tentativas válidas: 11  |  Tempo total: 0.07s
Score: {'uc_dias_distintos': 15, 'turma_dias_<=4_bonus': 2, 'turma_dias_excesso_penal': -2, 'consecutivos': 17, 'min_salas_penal': -2, 'total': 30}
Curso    | Aula(slot,sala)          | Aula2(slot,sala)        
----------------------------------------------------------------------
AST      | (19, SalaT       ) | (12, SalaT       )
GSI      | ( 3, SalaT       ) | ( 8, ONLINE      )
GUS      | (11, SalaT       ) | (20, SalaT       )
IA       | ( 2, SalaT       ) | (18, SalaT       )
IAAJ     | ( 4, SalaC       ) | (18, SalaC       )
IE       | (20, SalaC       ) | ( 9, Lab01       )
IM       | ( 4, Lab01       ) | ( 2, Lab01       )
ISC      | ( 5, SalaT       ) | (15, SalaT       )
ISI      | (16, Lab01       ) | ( 4, SalaT       )
PA       | ( 6, Lab01       ) | (17, Lab01       )
PAG      | (17, SalaC       ) | ( 3, SalaC       )
PDM      | ( 7, Lab01       ) | ( 1, Lab01       )
PS       | (15, SalaC       ) | (16, ONLINE      )
PVR      

## Conclusão

O projeto consistiu no desenvolvimento de um agente capaz de gerar automaticamente horários válidos e equilibrados. O problema foi tratado como um sistema de restrições (CSP), onde o agente procurou satisfazer todas as hard constraints, como disponibilidades dos docentes, limites diários e ausência de sobreposições, enquanto maximizava critérios de qualidade através de uma função de score. Atribuímos as salas numa segunda fase, com um algoritmo greedy que evita conflitos e respeita as exigências obrigatórias.
Os resultados demonstraram que o agente é capaz de gerar soluções válidas em menos de um segundo, respeitando todas as restrições obrigatórias e apresentando horários consistentes e sem conflitos de sala. A análise dos scores revelou uma boa distribuição de aulas em dias distintos e blocos consecutivos, bem como uma utilização eficiente das salas disponíveis. Apesar de pequenas penalizações em critérios secundários, como o número de dias com aulas por turma, o desempenho global do agente foi considerado satisfatório.